In [1]:
#https://developers.google.com/earth-engine/api_docs#eefeatureget
import ee
import datetime
import IPython.display
from IPython.display import Image
import bqplot
import ipywidgets
import pprint
pp = pprint.PrettyPrinter(indent=2) # for printing pretty idk what it is... print with pp.pprint(print stuff)
import pandas as pd
import geopandas as gpd
import matplotlib.dates as mdates
from matplotlib import dates
%matplotlib inline
from shapely.geometry import shape
import skimage
import traitlets
import dateutil.parser
import numpy as np
import ipyleaflet
from ipyleaflet import (LayersControl, basemaps, basemap_to_tiles, LayerGroup)

def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)


# Initialize
ee.Initialize()

#To-Do


In [63]:
# Dates
startTime = datetime.datetime(2013,1,1)
endTime = datetime.datetime(2019,5,1)

# AOI
ried = ee.FeatureCollection('users/tillmueller1990/ried_roi')
germany = ee.Collection.loadTable('USDOS/LSIB_SIMPLE/2017').filterMetadata('country_co','equals','GM')
germany_1 = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Germany'))

# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte();

# Paint all the polygon edges with the same number and width, display.
germany_outline = empty.paint(germany, 1, 3)


## Precipitaion Data (Radolan DWD)

In [19]:
#Load Radolan from Assest 
radolan = ee.ImageCollection('users/tillmueller1990/radolan/r_13_19')


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Soil Moisture

In [15]:
#Soil Moisture Datasets

#NASA-USDA Global Soil Moisture Data / 3 days / 2010- 2019 / 0.25 arc degrees / SMOS lvl2 integrated into two-layer palmer model
SMOS = ee.ImageCollection('NASA_USDA/HSL/soil_moisture')
surface_moisture = SMOS.select('ssm') #units mm, min=0 max=25
subsurface_moisture = SMOS.select('susm') #units: mm min=0, max=275
moisture_profile = SMOS.select('smp') #units: fraction, min0, max 1

#NASA-USDA SMAP Global Soil Moisture Data / 3 days / 2015 - 2019 / 0.25 arc degrees / SMAP level 3 + two-layer Palmer
SMAP = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture')

#GLDAS-2.1: Global Land Data Assimilation System / 3 hours / 0.25 arc degrees / 2000 - 2019 / 

#Datasets

## Corine Land Cover 

In [69]:
# Load last corine land cover image 
clc = ee.Image.load('COPERNICUS/CORINE/V18_5_1/100m/2012', -1)

# Mask areas where soil moisture measurements valid (farmland cat.:11-17)
clc_mask = clc.gte(11)and(clc.lte(17)) #binary map for updateMask
clc = clc.updateMask(clc_mask) #set mask for not Farmland

# Clip to extend of germany
clc = clc.clip(germany_1) 

# Feature Collection of clc farmland cat.: 11-17 with 350m resolution
clc_vector = clc.reduceToVectors(geometry=germany_1, crs=clc.projection(), scale=350, geometryType='polygon', eightConnected = False).limit(5000)
pp.pprint(clc_vector.size().getInfo())

5000


## Iterate over ImageCollection returning pandas dataframe 

In [127]:
images = ee.Image('NASA_USDA/HSL/soil_moisture/NASA_USDA_SM20150102_20150104')
pp.pprint(images.getInfo())

{ 'bands': [ { 'crs': 'EPSG:4326',
               'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0],
               'data_type': {'precision': 'float', 'type': 'PixelType'},
               'dimensions': [1440, 600],
               'id': 'ssm'},
             { 'crs': 'EPSG:4326',
               'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0],
               'data_type': {'precision': 'float', 'type': 'PixelType'},
               'dimensions': [1440, 600],
               'id': 'susm'},
             { 'crs': 'EPSG:4326',
               'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0],
               'data_type': {'precision': 'float', 'type': 'PixelType'},
               'dimensions': [1440, 600],
               'id': 'smp'},
             { 'crs': 'EPSG:4326',
               'crs_transform': [0.25, 0.0, -180.0, 0.0, -0.25, 90.0],
               'data_type': {'precision': 'float', 'type': 'PixelType'},
               'dimensions': [1440, 600],
               'id': 'ssma'}

In [128]:
#Geometrys, Regions
points = ee.FeatureCollection([
            ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797)),
            ee.Feature(ee.Geometry.Point(8.241221721890724,49.6585087644599)),
            ])
point = ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797))

#Time
start,end = datetime.datetime(2015,1,1),datetime.datetime(2016,1,1)
#Source Dataset
sm = surface_moisture.filterDate(start,end)

# Function Convert a FeatureCollection into a pandas DataFrame; Features is a list of dict with the output
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        #attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    pp.pprint(df.geometry)
    # Convert GeoJSON features to shape
    #WAS GENAU PASSIER HIER map lambda???
    df['geometry'] = map(lambda s: shape(), df.geometry)    
    return df

# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, pts):
    image = ee.Image(img_id)
    pp.pprint(image.getInfo()['properties']['ID'])
    #Ad reducer output to the Features in the collection.
    fc_image_red = image.reduceRegions(collection=pts, reducer=ee.Reducer.mean(), scale=350)
    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_red)
    # Add Date as Variable
    df_image_red['date'] = image.getInfo()['properties']['ID']
    return df_image_red

#List of image propertys 
sm_prop = surface_moisture.first().propertyNames().getInfo()
pp.pprint(sm_prop)

#Get propertie from every image to a List
sm_id = [] #empty list
for f in sm.getInfo()['features']: 
    image_id = str('NASA_USDA/HSL/soil_moisture/') + f['properties']['ID']#.encode('ascii', 'ignore')
    sm_id.append(image_id)
pp.pprint(sm_id[0:5])

#### Create Initial Pandas Dataframe
pp.pprint(sm_id[0])
df_all = extract_point_values(sm_id[0], points)
df_all = df_all.drop([0,1])

#### Iterate over all impages
for i in sm_id:
    df_all = df_all.append(extract_point_values(i, points))

#### Display Results
pp.pprint(df_all) 

#Get Pixel for points maybe polygons? Value must be a regional reducer over polygon ; 


[ 'system:time_start',
  'system:footprint',
  'system:time_end',
  'system:version',
  'system:id',
  'ID',
  'system:asset_size',
  'system:index',
  'system:bands',
  'system:band_names']
[ 'NASA_USDA/HSL/soil_moisture/NASA_USDA_SM20150102_20150104',
  'NASA_USDA/HSL/soil_moisture/NASA_USDA_SM20150105_20150107',
  'NASA_USDA/HSL/soil_moisture/NASA_USDA_SM20150108_20150110',
  'NASA_USDA/HSL/soil_moisture/NASA_USDA_SM20150111_20150113',
  'NASA_USDA/HSL/soil_moisture/NASA_USDA_SM20150114_20150116']
'NASA_USDA/HSL/soil_moisture/NASA_USDA_SM20150102_20150104'
'NASA_USDA_SM20150102_20150104'
0    {'type': 'Point', 'coordinates': [8.2348702509...
1    {'type': 'Point', 'coordinates': [8.2412217218...
Name: geometry, dtype: object


TypeError: 'module' object is not callable

## Soil Moisture from Sentinel 1

In [ ]:
#Derived from Sentinel 1 A/B 2014-10 - Present
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')


## Viewer

In [61]:
#Visualization Parameters
dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
germany_viz = GetTileLayerUrl(germany_outline.visualize())
#radolan_viz = GetTileLayerUrl(radolan.visualize(min=0,max=40,bands=['b1']))
clc_viz = GetTileLayerUrl(clc.visualize(min=0,max=7,palette=['FFE6FF','FFFFA8','FFFF00','E6E600','E68000','F2A64D','E6A600']))
clc_vector_viz = GetTileLayerUrl(clc_vector.draw(color='red').visualize())
#surface_moisture_viz = GetTileLayerUrl(surface_moisture.visualize(min=0,max=28,palette=['0300ff', '418504', 'efff07', 'efff07', 'ff0303']))

# Create layer group
layer_group = LayerGroup(layers=(ipyleaflet.TileLayer(url=clc_viz,name='Corine Land Cover 2012'), 
                                 ipyleaflet.TileLayer(url=germany_viz, name='german boundary'), 
                                 ipyleaflet.TileLayer(url=clc_vector_viz, name='clc_vector')
                                 #ipyleaflet.TileLayer(url=surface_moisture_viz, name='GSMD surface moisture')
                                ))

#Map options
center,zoom = [12.727661, 51.229822],4

#Interactive Visualizations
map1 = ipyleaflet.Map(layer =dark_matter_layer, center=center, zoom=zoom, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1.add_layer(layer_group)
map1.add_control(LayersControl())
map1


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [31]:
#Get Coordinates from Drawn Geometry
dc.last_draw

{'type': 'Feature',
 'properties': {'style': {'shapeOptions': {},
   'stroke': True,
   'color': '#3388ff',
   'weight': 4,
   'opacity': 0.5,
   'fill': True,
   'fillColor': None,
   'fillOpacity': 0.2,
   'clickable': True,
   'zIndexOffset': 2000,
   'icon': {'options': {}, '_initHooksCalled': True},
   'repeatMode': False}},
 'geometry': {'type': 'Point', 'coordinates': [12.727661, 51.229822]}}

In [ ]:
#Command Collection
.size().getInfo() # Get collection size
.bandNames().getInfo() # Get List with alls Bands from ee.Image not ImageCollection
.geometry().bounds().getInfo() # get Geometry of a Feature // Get bounding box of this geometry

#Vectorizing
var vectors = zones.addBands(nl2012).reduceToVectors({
  geometry: FeatureCollection,
  crs: image.projection(),
  scale: 1000,
  geometryType: 'polygon',
  eightConnected: false,
  labelProperty: 'zone',
  reducer: ee.Reducer.mean()
});

#Masking
.clip(feature)
var image = ee.Image
var mask = image.gte(2).and(lt(5))
var maskedImage = image.updateMask(mask)

#Visualizations
thumbnail_url = image.getThumbUrl({
    'bands' : '',
    'min' : ,
    'max' : ,
    'region' : .geometry().bounds().getInfo() #must be a geojson 
})
IPython.display.HTML('Thumnail URL: <a href={0}>{0}</a>'.format(thumbnail_url)) #create url to view
IPython.display.Image(url=thumbnail_url) # view direct in notebook

#Interactive Visualizations
import ipyleaflet
map1 = ipyleaflet.Map(zoom=3, layout={'height' : '400px'})
dc = ipyleaflet.DrawControl()
map1.add_control(dc)
map1
dc.last_draw # gives information about the last drawn polygon (coordinates etc.)

#Function to create a tile layer urlfrom an gee image object
def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

#style the image
tile_url = GetTileLayerUrl(image.visualize(min=0, max=3000, gamma=1.5, bands=['','','']))
map1.add_layer(ipyleaflet.TileLayer(url=tile_url))
#or create layer groups 

In [13]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import ee


def extract_time_series(lat, lon, start, end, product_name, band_name, sf):

    # Set up point geometry
    point = ee.Geometry.Point(lon, lat)

    # Obtain image collection for all images within query dates
    coll = ee.ImageCollection(product_name)\
        .filterDate(start, end)

    # Get list of images which correspond with the above
    images = [item.get('id') for item in coll.getInfo().get('features')]

    store = []
    date_store = []

    # Loop over all images and extract pixel value
    for image in images:

        im = ee.Image(image)
        projection = im.projection().getInfo()['crs']

        # Obtain date from timestamp in metadata
        date = dt.fromtimestamp(im.get("system:time_start").getInfo() / 1000.)
        date_store.append(np.datetime64(date))

        # Extract pixel value
        data = im.select(band_name)\
        .reduceRegion(ee.Reducer.first(),
                      point,
                      1,
                      crs=projection)\
        .get(band_name)

        store.append(data.getInfo())

    # Scale the returned data based on scale factor
    store = [x * sf if isinstance(x, int) else np.nan for x in store]

    # Convert output into pandas data frame
    df = pd.DataFrame(index=date_store, data=store, columns=[band_name])

    return print(df)


if __name__ == "__main__":

    ee.Initialize()

    latitude = 49.808
    longitude = 8.234
    start_date = '2018-01-01'
    end_date = '2018-01-31'
    product = 'NASA_USDA/HSL/soil_moisture' 
    band = 'ssm'
    scale_factor = 1

    # Extract data and obtain pd.DataFrame
    output = extract_time_series(latitude,
                                 longitude,
                                 start_date,
                                 end_date,
                                 product,
                                 band,
                                 scale_factor)


                     ssm
2018-01-01 01:00:00  NaN
2018-01-04 01:00:00  NaN
2018-01-07 01:00:00  NaN
2018-01-10 01:00:00  NaN
2018-01-13 01:00:00  NaN
2018-01-16 01:00:00  NaN
2018-01-19 01:00:00  NaN
2018-01-22 01:00:00  NaN
2018-01-25 01:00:00  NaN
2018-01-28 01:00:00  NaN


In [4]:
from ipygee import *


In [106]:
# ==========================================================================
# Function to Convert Feature Classes to Pandas Dataframe
# Adapted from: https://events.hpc.grnet.gr/event/47/material/1/12.py
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    df['geometry'] = map(lambda s: shape(s), df.geometry)    
    return df

# ==========================================================================
# Function to iterate over image collection, returning a pandas dataframe
def extract_point_values(img_id, pts):
    image = ee.Image(img_id)

    fc_image_red = image.reduceRegions(collection=pts,
                                  reducer=ee.Reducer.mean(),
                                  scale=30)

    # Convert to Pandas Dataframe
    df_image_red = fc2df(fc_image_red)

    # Add Date as Variable
    df_image_red['date'] = image.getInfo()['properties']['DATE_ACQUIRED']

    return df_image_red

# ==========================================================================
#### Make Points
points = ee.FeatureCollection([
            ee.Feature(ee.Geometry.Point(8.234870250943459,49.80831604635797)),
            ee.Feature(ee.Geometry.Point(8.241221721890724,49.6585087644599)),
            ])

#### Load Raster
sm = surface_moisture

#### Make list of image IDs
sm_id = []
for f in sm.getInfo()['features']:
  image_id = f['properties']['LANDSAT_SCENE_ID'].encode('ascii', 'ignore')
  image_id = 'LANDSAT/LC8_L1T/' + image_id
  l8_id.append(image_id)

#### Create Initial Pandas Dataframe
df_all = extract_point_values(l8_id[0], points)
df_all = df_all.drop([0,1])

#### Iterate over all impages
for i in l8_id:
    df_all = df_all.append(extract_point_values(i, points))

#### Display Results
df_all    

KeyError: 'LANDSAT_SCENE_ID'